In [ ]:
import operator

import httpx
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
import getpass

API_KEY = getpass.getpass()

In [4]:
proxies = {
            'http://': '',
            'https://': '',
        }

In [5]:
openai_llm = ChatOpenAI(api_key=API_KEY, model='gpt-3.5-turbo', http_client=httpx.Client(proxies=proxies))

In [ ]:
local_llm = ChatOllama(model='aya:35b-23-q2_K', base_url='http://localhost:11434', streaming=True)

In [6]:
_atoms_generation_system = """Задача: На основе приведенного текста сгенерируйте список атомарных утверждений, которые отражают ключевые идеи и факты из текста. Каждое утверждение должно быть простым, недвусмысленным и выражать только одну мысль.

Формат ответа:
- 
- 
- 
...

Постарайтесь выделить 5-10 ключевых атомарных утверждений из текста. Если текст короткий и простой, меньшего количества может быть достаточно. Для длинных и сложных текстов может потребоваться больше утверждений. Ваша цель - четко и лаконично передать суть исходного текста."""

In [7]:
atoms_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", _atoms_generation_system),
        ("human", "{fragment}"),
    ]
)

In [8]:
atoms_chain = atoms_prompt | openai_llm | StrOutputParser()

In [9]:
result = atoms_chain.invoke({"fragment": """Диплом о профессиональной переподготовке
Документ подтверждает, что слушатель успешно окончил программу профессиональной переподготовки и получил право на ведение профессиональной деятельности. Диплом о профессиональной переподготовке является документом о квалификации и вносится в Федеральный реестр документов об образовании.

Диплом выдаём только на русском языке, в печатном виде.

Можем его выдать, если вы:

оплатили обучение после 3 декабря 2019 года;
укладываетесь в сроки, на которые рассчитана программа, и 6 месяцев сверху. Срок отсчитывается с даты оплаты обучения;
имеете среднее или высшее профессиональное образование — подойдут диплом СПО или ВО. Если вы ещё не закончили СУЗ или ВУЗ, подойдёт справка из образовательного учреждения;
при зачислении предоставили пакет документов — паспорт, диплом, СНИЛС, сведения о смене ФИО (при наличии), сведения о признании иностранного диплома (при наличии);
сдали все промежуточные аттестации до даты итоговой аттестации;
успешно сдали итоговую аттестацию.
Итоговой аттестацией может быть:

итоговый экзамен;
защита проекта;
другие формы итоговой аттестационной работы.
Мы подготовим диплом в течение 30 дней от даты итоговой аттестации. Чтобы его получить, обратитесь к куратору."""})

In [ ]:
with open('result.txt', 'w') as f:
    f.write(result)

In [6]:
_questions_generation_system = """Задача: Сгенерируйте 5-10 вопросов, ответом на которые  будет приведенное утверждение. Вопросы должн быть сформулированы так, чтобы утверждение было ответом на них.

Формат ответа:
- 
- 
- 

Постарайтесь написать 5-10 вопрсосов на основе текста. Если текст короткий и простой, меньшего количества может быть достаточно. Для длинных и сложных текстов может потребоваться больше вопросов. В ответ напиши только вопросы и ничего более."""

In [7]:
questions_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", _questions_generation_system),
        ("human", "{atom}"),
    ]
)

In [8]:
questions_chain = questions_prompt | openai_llm | StrOutputParser()

In [36]:
questions_chain.invoke({"atom": """Диплом будет подготовлен в течение 30 дней после успешной прохождения итоговой аттестации."""})

In [11]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [33]:
def format_docs(_docs):
    formated_docs = [f"text_id: {_doc.metadata.get('answer_class')}\ncontent:\n{_doc.page_content}" for _doc in _docs]
    print(formated_docs)
    return "\n\n".join(formated_docs[:5])

In [18]:
def remove_numbers(_text):
    lines = _text.split('\n')
    _result = [line.strip()[2:] for line in lines]
        
    return _result

In [19]:
from langchain_core.documents import Document

In [21]:
def get_sub_docs_only_questions(_doc):
    base_content = _doc.page_content
    # row_atoms = atoms_chain.invoke({"fragment": base_content})
    # list_atoms = remove_numbers(row_atoms)
    row_questions = questions_chain.invoke({"atom": base_content})
    list_questions = remove_numbers(row_questions)
    new_docs = [Document(page_content=question, metadata=_doc.metadata) for question in list_questions]
    # docs_to_return = [] 
    # 
    # for atom in list_atoms:
    #     row_questions = questions_chain.invoke({"atom": atom})
    #     list_questions = remove_numbers(row_questions)
    #     new_docs = [Document(page_content=question, metadata=_doc.metadata) for question in list_questions]
    #     
    #     docs_to_return.extend(new_docs)
        
    return new_docs

In [16]:
def get_sub_docs_full_chain(_doc):
    base_content = _doc.page_content
    row_atoms = atoms_chain.invoke({"fragment": base_content})
    list_atoms = remove_numbers(row_atoms)
    docs_to_return = [] 
    
    for atom in list_atoms:
        row_questions = questions_chain.invoke({"atom": atom})
        list_questions = remove_numbers(row_questions)
        new_docs = [Document(page_content=question, metadata=_doc.metadata) for question in list_questions]
        
        docs_to_return.extend(new_docs)
        
    return docs_to_return

In [ ]:
get_sub_docs_full_chain(Document(page_content="""Диплом о профессиональной переподготовке
Документ подтверждает, что слушатель успешно окончил программу профессиональной переподготовки и получил право на ведение профессиональной деятельности. Диплом о профессиональной переподготовке является документом о квалификации и вносится в Федеральный реестр документов об образовании.

Диплом выдаём только на русском языке, в печатном виде.

Можем его выдать, если вы:

оплатили обучение после 3 декабря 2019 года;
укладываетесь в сроки, на которые рассчитана программа, и 6 месяцев сверху. Срок отсчитывается с даты оплаты обучения;
имеете среднее или высшее профессиональное образование — подойдут диплом СПО или ВО. Если вы ещё не закончили СУЗ или ВУЗ, подойдёт справка из образовательного учреждения;
при зачислении предоставили пакет документов — паспорт, диплом, СНИЛС, сведения о смене ФИО (при наличии), сведения о признании иностранного диплома (при наличии);
сдали все промежуточные аттестации до даты итоговой аттестации;
успешно сдали итоговую аттестацию.
Итоговой аттестацией может быть:

итоговый экзамен;
защита проекта;
другие формы итоговой аттестационной работы.
Мы подготовим диплом в течение 30 дней от даты итоговой аттестации. Чтобы его получить, обратитесь к куратору.""", metadata={}))

In [22]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Chroma

In [26]:
loader = CSVLoader(
    file_path='./answer_class.csv',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
    },
    metadata_columns=['answer_class']
)

docs = loader.load()

for doc in docs:
    doc.page_content = doc.page_content.replace('answer: ', '')

In [27]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")

In [28]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embedding_function
)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [29]:
sub_docs = []
for i, doc in enumerate(docs):
    print(i)
    _id = doc_ids[i]
    _sub_docs = get_sub_docs_only_questions(doc)
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [30]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [54]:
retriever.vectorstore.similarity_search("", k=100)

In [31]:
retriever.search_kwargs = {"k": 15}

In [32]:
import pandas as pd

In [26]:
import operator

def predict_answer(question):
    _predicted_answer = retriever.vectorstore.similarity_search(question, k=15)[0].metadata.get('answer_class')
    return int(_predicted_answer)

In [28]:
def predict_answers(question):
    _predicted_answers = retriever.vectorstore.similarity_search(question, k=5)
    
    return [int(ans.metadata.get('answer_class')) for ans in _predicted_answers]

In [35]:
# Читаем CSV файл в DataFrame
df = pd.read_csv('test_data.csv')

df['answer_class'] = df['answer_class'].apply(int)
# Применяем функцию predict_answer к каждому вопросу и сохраняем результаты в новый столбец
df['predicted_answer'] = df['question'].apply(predict_answer_with_llm)

# Считаем процент совпадений предсказанных ответов с правильными ответами
accuracy = (df['predicted_answer'] == df['answer_class']).mean() * 100

# Сохраняем результаты обратно в CSV файл
df.to_csv('data_with_predictions.csv', index=False)

print(f"Accuracy: {accuracy:.2f}%")

In [29]:
df = pd.read_csv('test_data.csv')

correct_in_top5 = 0
correct_present = 0
total_tests = len(df)

for _, row in df.iterrows():
    question = row['question']
    
    predicted_answers = predict_answers(question)
    answer = row['answer_class']
    
    if answer in predicted_answers:
        correct_present += 1
        
        if answer in predicted_answers[:5]:
            correct_in_top5 += 1
    

top5_percentage = correct_in_top5 / total_tests * 100
present_percentage = correct_present / total_tests * 100

print(f"Верный ответ присутствует в топ-10 в {present_percentage:.2f}% тестов")
print(f"Верный ответ в топ-5 в {top5_percentage:.2f}% тестов")